#### Spark

In [ ]:
df = spark.createDataFrame(
    data=[
        ("LEMON", "Yellow",), 
        ("LIME", "Green",)
    ], schema=["fruit", "color"])

df.createOrReplaceTempView("fruits_table")

display(df)

In [ ]:
from flypipe.node import node
from flypipe.datasource.spark import Spark

@node(
    type="pyspark",
    dependencies=[Spark("fruits_table").select("fruit")]
)
def my_fruits(fruits_table):
    return fruits_table

df = my_fruits.run(spark)
display(df)              

displayHTML(my_fruits.html())

#### Adding your own data source

In [ ]:
import os
import re
import pandas as pd

from flypipe.node import node
from flypipe.node_type import NodeType


def CSV(path_csv):
    @node(
        type="pandas",
        description=f"Loading CSV file at {path_csv}",
        tags=["datasource", "csv"]
    )
    def load_csv():
        return pd.read_csv(open(path_csv, 'r'))

    file_name = os.path.basename(path_csv)
    file_name = re.sub("[^\da-zA-Z]", "_", file_name)
    key = re.sub("[^\da-zA-Z]", "_", path_csv)

    load_csv.function.__name__ = file_name
    load_csv.key = key
    load_csv.node_type = NodeType.DATASOURCE
    return load_csv

##### Testing

In [ ]:
import pytest
import pandas as pd
from pyspark_test import assert_pyspark_df_equal
from pandas.testing import assert_frame_equal

from flypipe.node import node
from flypipe.datasource.spark import Spark
from flypipe.schema.column import Column
from flypipe.schema.schema import Schema
from flypipe.schema.types import Long

#Fixtures
@pytest.fixture(scope="function")
def dummy_df():
    import pandas as pd
    
    #dummy dataframe
    df = pd.DataFrame(columns=["color", "fruit"])
    df.loc[0] = ["yellow", "lemon"]
    df.loc[0] = ["green", "lime"]
    
    #save dataframe 
    df.to_csv("/tmp/test.csv", index=False)
    
    return df


class TestPyspark:
    
    def test_(self, dummy_df):
        """
        Test a CSV datasource
        """
        
        @node(
            type="pandas",
            dependencies=[
                CSV("/tmp/test.csv").select(["fruit", "color"])
            ]
        )
        def t1(test_csv):
            return test_csv

        df = t1.run()
        assert_frame_equal(dummy_df, df)
        
import ipytest
ipytest.run()